#  Entrenamiento del Modelo CNN — PlantVillage
Este notebook continúa después de la **Preparación de Datos**. Aquí entrenaremos una **CNN simple desde cero** con el dataset de hojas de cultivos.

## 🔹 Objetivo
Entrenar un modelo de red neuronal convolucional (CNN) que clasifique hojas en distintas enfermedades o sanas.


In [ ]:
# 1️ Importar librerías
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
print('TensorFlow version:', tf.__version__)

In [ ]:
# 2️ Configuración de rutas y parámetros básicos
base_dir = r"C:\\Users\\Admin\\Downloads\\archive (1)\\plantvillage dataset\\color"  # Ajusta si usas otra carpeta
img_size = (128, 128)
batch_size = 32
epochs = 15

# Verificar clases
classes = sorted(os.listdir(base_dir))
num_classes = len(classes)
print(f"Se detectaron {num_classes} clases:")
print(classes[:10])

In [ ]:
# 3️ Generadores de imágenes para entrenamiento y validación
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print(f"Total de imágenes de entrenamiento: {train_gen.samples}")
print(f"Total de imágenes de validación: {val_gen.samples}")

In [ ]:
# 4️ Definir arquitectura CNN desde cero
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 5️ Entrenamiento del modelo
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('modelo_cultivos.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=callbacks
)

In [ ]:
# 6️ Gráficos de entrenamiento
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del modelo')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida del modelo')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 7️ Evaluación del modelo
val_loss, val_acc = model.evaluate(val_gen)
print(f"Precisión de validación: {val_acc*100:.2f}%")

In [ ]:
# 8️ Matriz de confusión y reporte
y_true = val_gen.classes
y_pred = np.argmax(model.predict(val_gen), axis=1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=False, cmap='Blues')
plt.title('Matriz de confusión')
plt.show()

print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))

 **Modelo guardado:** `modelo_cultivos.h5`

Este archivo puede cargarse luego con:
```python
from tensorflow.keras.models import load_model
modelo = load_model('modelo_cultivos.h5')
```
Y usarse para hacer predicciones sobre nuevas imágenes de hojas.
